# 데이터 로드

In [77]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from dotenv import load_dotenv
import os
import urllib.request
from datetime import datetime
import re
import urllib.parse

load_dotenv()
client_id = os.getenv('Client_ID')
client_secret = os.getenv('Client_Secret')

def getnews_data(word, display=100):
    
    encoded_query = urllib.parse.quote(word)  # 한글을 URL 인코딩
    url = 'https://openapi.naver.com/v1/search/news.json?query={}&display={}'.format(encoded_query, display)
    
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            items = response.json()['items']
            news_list = []
            
            for idx, item in enumerate(items):
                # HTML 태그 제거
                link = item.get('link')
                title = re.sub('<.*?>', '', item.get('title', ''))
                title = title.replace('&quot;', '"').replace('&amp;', '&')
                description = re.sub('<.*?>', '', item.get('description', ''))
                description = description.replace('&quot;', '"').replace('&amp;', '&')
                
                news_data = {
                    'no': idx + 1,
                    'title': title, #뉴스 제목
                    'link': item.get('link', ''), #뉴스 링크
                    'description': description, #뉴스 요약
                    'pubDate': item.get('pubDate', ''), #발행 날짜
                    'collected_time': datetime.now().strftime("%Y-%m-%d %H:%M:%S") #데이터 수집 기간
                }
                news_list.append(news_data)
                
                # 진행상황 출력
                #print(f"{idx + 1}. {title}")
            
            print(f"'{word}' 관련 뉴스 {len(news_list)}개 수집 완료")
            news_df = pd.DataFrame(news_list)
            return news_df
            
        else:
            print(f"API 오류: {response.status_code}")
            return pd.DataFrame()
            
    except Exception as e:
        print(f"오류 발생: {e}")
        return pd.DataFrame()

In [78]:
getnews_data('서울날씨', 100)



'서울날씨' 관련 뉴스 100개 수집 완료


,no,title,link,description,pubDate,collected_time
0,1,"진교훈 강서구청장, 101세 생신 맞은 어르신에 큰절",https://n.news.naver.com/mnews/article/016/000...,[강서구 제공] 진교훈 서울 강서구청장이 15일 ‘백세 어르신 축하연 및 효잔치’ ...,"Tue, 15 Jul 2025 16:34:00 +0900",2025-07-15 16:37:04
1,2,[내일날씨] 전국 비 소식 계속…중부지역 곳곳 '물폭탄',https://www.asiatoday.co.kr/view.php?key=20250...,특히 중부지역을 중심으로 좁은 비구름대가 형성돼 많은 비가 쏟아지겠다.16∼17일 ...,"Tue, 15 Jul 2025 16:30:00 +0900",2025-07-15 16:37:04
2,3,"진교훈 강서구청장, 101세 어르신께 큰절…""존경하고 사랑합니다""",https://www.pinpointnews.co.kr/news/articleVie...,제공=강서구청 진교훈 서울 강서구청장이 15일 지역 어르신들을 위한 '백세 어르신 ...,"Tue, 15 Jul 2025 16:28:00 +0900",2025-07-15 16:37:04
3,4,[날씨] 내일~모레 전국 강한 비…중부 최대 200mm,https://n.news.naver.com/mnews/article/422/000...,"한낮에는 서울, 대구, 부산은 28도 보이며 비로 인해 폭염이 한풀 꺾인 날이 유지...","Tue, 15 Jul 2025 16:28:00 +0900",2025-07-15 16:37:04
4,5,"중학생이 만드는 명품 부채, 부채 둘이 하나가 되는 쌍죽선",https://n.news.naver.com/mnews/article/047/000...,"지난 12일 무더운 날씨에도, 김복남 명인이 자신의 공방에서 부채를 만들고 있었다....","Tue, 15 Jul 2025 16:10:00 +0900",2025-07-15 16:37:04
...,...,...,...,...,...,...
95,96,'폭염이 재난이 된 시대' 일상도 위협,http://www.seongjuro.co.kr/news/view.php?idx=5...,고온다습한 날씨에 식중독 발생 가능성도 높아지면서 보건소는 가천·금수강산면 등 행락...,"Tue, 15 Jul 2025 09:14:00 +0900",2025-07-15 16:37:04
96,97,[오늘의 날씨] 전국 흐리고 비···그치면 고온다습,https://www.smartfn.co.kr/news/articleView.htm...,"예상 강수량은 ▲서울·인천·경기, 강원 영동 남부·강원 영서, 대전·세종·충남, 광...","Tue, 15 Jul 2025 09:06:00 +0900",2025-07-15 16:37:04
97,98,[오늘날씨] 전국 대체로 흐리고 곳곳 비… 낮 최고 31도,http://www.shinailbo.co.kr/news/articleView.ht...,"오전 5시 기준 주요 지역의 기온은 서울 21.6도, 인천 21.0도, 수원 20....","Tue, 15 Jul 2025 09:04:00 +0900",2025-07-15 16:37:04
98,99,16~17일 최대 150㎜·서울 80㎜ '물폭탄'…시간당 50㎜ 퍼붓는 곳도 [내일...,https://n.news.naver.com/mnews/article/421/000...,"16~17일 예상 강수량은 서울과 인천, 경기 북부에 30~80㎜, 경기 남부에 5...","Tue, 15 Jul 2025 09:00:00 +0900",2025-07-15 16:37:04


# 필터링 함수

In [ ]:
def filter_weather_news(news_df):
        """날씨 관련 뉴스만 필터링"""
        weather_keywords = ['날씨', '기온', '온도', '비', '눈', '바람', '습도', '기상', '예보', '폭염', '한파', '태풍']
        
        filtered_news = []
        for _, row in news_df.iterrows():
            title = row['title'].lower()
            
            # 제목에 날씨 키워드가 포함된 경우만
            if any(keyword in title for keyword in weather_keywords):
                filtered_news.append(row)
        
        return pd.DataFrame(filtered_news)

# 중복 제거 & 데이터 저장(excel) & 업데이트

In [ ]:
from pathlib import Path
from datetime import datetime
def update_and_save_news(word, display=100, file_path=None, max_rows=100):
    #1. 파일 경로 설정
    if file_path is None:
        file_path = f"news_data/{word}_news.xlsx"

    print(f"=== '{word}' 뉴스 업데이트 시작 ===")
    print(f"수집 개수: {display}개")
    print(f"저장 경로: {file_path}")
    print("-" * 50)

    #2. 기존 데이터 로드
    existing_df = pd.DataFrame()
    if Path(file_path).exists():
        try:
            existing_df = pd.read_excel(file_path)
            print(f"기존 데이터 로드 완료: {len(existing_df)} 개")
        except Exception as e:
            print(f"기존 데이터 로드 오류 : {e}")
    else:
        print(f"기존 데이터 로드 실패 : {file_path} 파일이 존재하지 않음. 새로 생성 요망")

    #3. 새로운 뉴스 데이터 수집
    crawler = getnews_data(word, display)
    new_news_df = crawler.getnews_data(word, display)  # 메서드 호출
    new_news_df = filter_weather_news(new_news_df) #뉴스 필터링
    if new_news_df.empty:
        print("새로운 데이터 수집에 실패했습니다.")
        return False
    
    # 4. 중복 제거
    if existing_df.empty:
        new_df_filtered = new_news_df
        duplicates_count = 0
    else:
        new_df_filtered = new_news_df[~new_news_df['link'].isin(existing_df['link'])]
        duplicates_count = len(new_news_df) - len(new_df_filtered)
    
    print(f"중복 제거: {duplicates_count}개")
    print(f"새로운 뉴스: {len(new_df_filtered)}개")
    
    # 5. 새로운 뉴스가 없으면 종료
    if new_df_filtered.empty:
        print("새로운 뉴스가 없습니다.")
        return False
    
    # 6. 데이터 병합 (새로운 뉴스를 맨 위에 추가)
    if existing_df.empty:
        final_df = new_df_filtered
    else:
        final_df = pd.concat([new_df_filtered, existing_df], ignore_index=True)
    
    # 7. no 컬럼 재정렬
    final_df['no'] = range(1, len(final_df) + 1)
    
    # 8. 엑셀 파일로 저장
    try:
        final_df.to_excel(file_path, index=False)
        
        print(f"'{word}' 뉴스 데이터가 '{file_path}'에 저장되었습니다.")
        print(f"새로 추가된 뉴스: {len(new_df_filtered)}개")
        print(f"전체 뉴스 개수: {len(final_df)}개")
        
        return True
        
    except Exception as e:
        print(f"파일 저장 오류: {e}")
        return False

    

In [86]:
update_and_save_news('서울날씨뉴스', 100)

=== '서울날씨뉴스' 뉴스 업데이트 시작 ===
수집 개수: 100개
저장 경로: news_data/서울날씨뉴스_news.xlsx
--------------------------------------------------
기존 데이터 로드 완료: 103 개
'서울날씨뉴스' 관련 뉴스 100개 수집 완료
중복 제거: 100개
새로운 뉴스: 0개
새로운 뉴스가 없습니다.


False